<a href="https://colab.research.google.com/github/miguelsalcedo01/tortoise-tts/blob/VoiceAI/Copy_of_tortoise_tts_origCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [1]:
!pip3 install -U scipy
!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install transformers==4.19.0
!pip3 install -r requirements.txt
!pip3 install einops==0.5.0
!pip3 install rotary_embedding_torch==0.1.5
!pip3 install unidecode==1.3.5
!python3 setup.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (442/442), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 1481 (delta 397), reused 392 (delta 392), pack-reused 1039
Receiving objects: 100% (1481/1481), 53.55 MiB | 32.05 MiB/s, done.
Resolving deltas: 100% (615/615), done.
/content/tortoise-tts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 25.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

import locale
def getpreferredencoding(do_setlocale = True):
    return "utf-8"
locale.getpreferredencoding = getpreferredencoding


# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

In [4]:
#optionally, upload use your own voice by running the net two cells. I recommend 
#you upload at least 5 audio clips. they must be a wav file, 22020 hz and 6-10 secs long.
CUSTOM_VOICE_NAME = "teena"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
    with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
       f.write(file_data)
    

Saving audion_seg_1.wav to audion_seg_1.wav
Saving audion_seg_2.wav to audion_seg_2.wav
Saving audion_seg_3.wav to audion_seg_3.wav
Saving audion_seg_4.wav to audion_seg_4.wav
Saving audion_seg_5.wav to audion_seg_5.wav
Saving audion_seg_6.wav to audion_seg_6.wav
Saving audion_seg_7.wav to audion_seg_7.wav
Saving audion_seg_8.wav to audion_seg_8.wav
Saving audion_seg_9.wav to audion_seg_9.wav
Saving audion_seg_10.wav to audion_seg_10.wav


In [8]:
# This is the text that will be spoken.
text = "Hi Miguel It's Teena.. I've missed you Miguel. I miss you sliding your long fat cock into my ass"

# Here's something for the poetically inclined.. (set text=)
"""
Hi Miguel you hear that? Wait what is that? It's Liquid sunshine! Yay! I've missed you Miguel! "You miss sliding your long ass cock in my ass don't you..
"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

In [10]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('tortoise/voices/freeman/1.wav')

NotImplementedError: ignored

In [ ]:
# Pick one of the voices from the output above
voice = 'freeman'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [9]:
# Tortoise can also generate speech using with custom voice files. 
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [01:33<00:00,  5.86s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 16/16 [00:45<00:00,  2.85s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [01:28<00:00,  4.53it/s]


In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..